In [1]:
man = [1,0,0,0,1,1,0,0,0]
man

[1, 0, 0, 0, 1, 1, 0, 0, 0]

In [374]:
### WIDTH = 10;

FP16_MAN_LEN = 10
FP16_EXP_LEN = 5
FP16_EXP_BIAS = 2**(FP16_EXP_LEN-1)-1         # 15
FP32_MAN_LEN = 23
FP32_EXP_LEN = 8
FP32_EXP_BIAS = 2**(FP32_EXP_LEN-1)-1         # 127
FP64_MAN_LEN = 52
FP64_EXP_LEN = 11
FP64_EXP_BIAS = 2**(FP64_EXP_LEN-1)-1         # 1023

FMT_FP16  = [1,FP16_MAN_LEN,FP16_EXP_BIAS,FP16_EXP_LEN]
FMT_FP32  = [1,FP32_MAN_LEN,FP32_EXP_BIAS,FP32_EXP_LEN]
FMT_FP32N = [3,FP32_MAN_LEN,FP32_EXP_BIAS,FP32_EXP_LEN]
#            \._ 1: has_denorm 3: force_denorm 

# CAUTION: Python List and Binary indexes left-to-right
# Call list reverse "a.reverse()"" before indexing in verilog fashion

# Useful Lambda Functions

# Convert Binary String (without '0b') to Binary Array
strdnum  = lambda s    : list(int(x) for x in s)
# Convert Binary Array to Binary String (without '0b')
numdstr  = lambda n    : "".join(str(x) for x in n)
# Convert Binary Array to Integer
bin2int  = lambda b    : int(numdstr(b),2)
# Convert Integer to Binary Array
int2bin  = lambda i, w : strdnum(bin(i)[2:].zfill(w))
# Reverse Indexing
indexrev = lambda a, i : a[len(a)-1-i]
# Check If an Array Only Has One Variety of Element
is_all   = lambda a, e : (a.count(e)==len(a))

# BINARY TOOLS
xor      = lambda a, b       : int((a and not b) or (not a and b))
# Word size is strictly defined by the input vectors
binadd   = lambda ba, bb, bci: int2bin(bin2int(ba)+bin2int(bb)+bci,max(len(ba),len(bb))+1)
# Sign Extension
binsext  = lambda b, ws      : [b[0]]*(ws-len(b))+b;
# Negate Binary Value; word size unchanged
binneg   = lambda b          : binadd(list(1-i for i in b),[0],1)[1:len(b)+1];

# Convert binary array to Python float value
def fpdecode(fmt,pack):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    sgn = pack[0]
    exp = pack[1:exp_len+1]
    man = pack[exp_len+1:]
    
    if (is_all(exp,0) and has_denorm) or force_denorm:
        # DENORM
        print("fpdecode: DENORM")
        t = float(bin2int(man))*2**(-man_len+1)
        t = t*(2**(-exp_bias))
    
    # SPECIAL numbers are not dealt-with in this implementation
    elif (is_all(exp,1)) :
        print("fpdecode: ERROR: SPECIAL NUMBER");
        return float("nan")
    else:
        # NORM
        t = float(bin2int([1] + man))*2**(-man_len)
        t = t*(2**(float(bin2int(exp))-exp_bias))
        
    if sgn == 1:
            t = t*-1
    return t

# Conver float value to binary array
import struct
from math import log
def double2binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!d', num))

def half2binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!e', num))

# Convert Python float (double) value to binary array in fmt
# FP16 doesn't need this function:
#    just call half2binary to get a binary string, 
#    and then convert it to a binary array with strdnum(s)
def fpencode(fmt,num):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    a = double2binary(num);
    sgn = a[0]
    exp64 = a[1:FP64_EXP_LEN+1]
    man64 = a[FP64_EXP_LEN+1:]
    
    # Convert Exponent
    exp64i = bin2int(exp64)
    exp = exp64i - FP64_EXP_BIAS + exp_bias
    if is_all(exp64,'0'):
        # Zeros
        return [int(sgn)] + [0]*exp_len + [0]*man_len
    if is_all(exp64,'1'):
        # Infinity
        return [int(sgn)] + [1]*exp_len + [0]*man_len
    if abs(exp) >= 2**(exp_len)-1:
        # Overflow, not implemented yet
        print("ERROR: EXP OVERFLOW");
        return [int(sgn)] + [1]*exp_len + [0]*man_len
    if(exp>0):
        exp = int2bin(exp,FP64_EXP_LEN)[FP64_EXP_LEN - exp_len:] 
        # Convert Mantissa
        man = strdnum(man64[0:man_len])
    else:
        # Denorm
        man = ([0]*abs(exp)+[1]+strdnum(man64))[0:man_len]
        exp = int2bin(0,exp_len)
    return [int(sgn)] + exp + man;


man = [0,0,0,0,0,0,0,0,0,0]
exp = [0,0,0,0,1]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [1,1,1,1,1,1,1,1,1,1]
exp = [0,0,0,0,0]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [0,0,0,0,0,0,0,0,0,1]
exp = [0,0,0,0,0]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [0,0,0,0,0,0,0,0,0,0]
exp = [0,0,0,0,0]
pack = [1] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

6.103515625e-05
fpdecode: DENORM
6.097555160522461e-05
fpdecode: DENORM
5.960464477539063e-08
fpdecode: DENORM
-0.0


In [507]:
[0]*10

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [224]:
# FP16 Back-and-forth Conversion Test
# https://en.wikipedia.org/wiki/Half-precision_floating-point_format

def fp16verify(s, desc):
    t = fpdecode(FMT_FP16,strdnum(s));
    # print(t);
    k = numdstr(fpencode(FMT_FP16,t));
    if k==s:
        print('- pass -', desc)
    else:
        print('- FAIL - ', desc)
    print('Expected:\t', s);
    print('Result  :\t', k);

fp16verify('0000000000000001','Smallest Positive Subnormal')
fp16verify('0000001010101010','Subnormal')
fp16verify('0000001111111111','Largest Subnormal')
fp16verify('0000010000000000','Smallest Positive Normal')
fp16verify('0011010101010101','Nearest Value to 1/3')
fp16verify('0011101111111111','Largest Number Less Than One')
fp16verify('0011110000000000','One')
fp16verify('0011110000000001','Smallest Number Larger Than One')
fp16verify('0111101111111111','Largest Normal Number')
fp16verify('0011010101010101','Nearest Value to 1/3')
fp16verify('0111110000000000','Infinity')
fp16verify('1000000000000000','Negative Zero')
fp16verify('0000000000000000','Positive Zero')
fp16verify('1100000000000000','-2')
fp16verify('1111111111111111','-Infinity')

fpdecode: DENORM
- pass - Smallest Positive Subnormal
Expected:	 0000000000000001
Result  :	 0000000000000001
fpdecode: DENORM
- pass - Subnormal
Expected:	 0000001010101010
Result  :	 0000001010101010
fpdecode: DENORM
- pass - Largest Subnormal
Expected:	 0000001111111111
Result  :	 0000001111111111
- pass - Smallest Positive Normal
Expected:	 0000010000000000
Result  :	 0000010000000000
- pass - Nearest Value to 1/3
Expected:	 0011010101010101
Result  :	 0011010101010101
- pass - Largest Number Less Than One
Expected:	 0011101111111111
Result  :	 0011101111111111
- pass - One
Expected:	 0011110000000000
Result  :	 0011110000000000
- pass - Smallest Number Larger Than One
Expected:	 0011110000000001
Result  :	 0011110000000001
- pass - Largest Normal Number
Expected:	 0111101111111111
Result  :	 0111101111111111
- pass - Nearest Value to 1/3
Expected:	 0011010101010101
Result  :	 0011010101010101
fpdecode: ERROR: SPECIAL NUMBER
- pass - Infinity
Expected:	 0111110000000000
Result  :	 

In [24]:
bin2int('0101010010101')
import math
def convexp(exp, exp_len, exp_bias):
    exp = bin2int(exp) - FP64_EXP_BIAS + exp_bias
    if (math.log(abs(exp))/math.log(2)) >= exp_len:
        # Overflow, not implemented yet
        print("ERROR: EXP OVERFLOW");
        return []
    exp = int2bin(exp,FP64_EXP_LEN)[FP64_EXP_LEN - exp_len:]
    return exp
    
convexp('01111110001',5,15)

[0, 0, 0, 0, 1]

In [442]:
# Note: For simplicity, ALL INTERNAL DATA is encoded in NON-NORMALIZED FORMAT
# that is: no hidden ones at the beginning of each encoded number, all digits are explicitly listed

# Show Verbose Debug Messages
DEBUGMODE = 0;

# This block locates at the beginning of the FIR pipeline
# Convert Normal FP16 Numbers to Internal Non-Normalized FMT (IFMT)
def fp16_to_ifmt(fmt, fp16):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    # Convert and unpack
    pack = strdnum(half2binary(fp16))
    sgn = pack[0]
    exp = pack[1:FP16_EXP_LEN+1]
    man = pack[FP16_EXP_LEN+1:]

    # Add the new bias value
    expn = bin2int(exp) - FP16_EXP_BIAS + exp_bias

    # Deal with Denorm
    if(is_all(exp,0)):
        if(DEBUGMODE):
            print("DENORM");
        man = man + [0]*(man_len - FP16_MAN_LEN)
    else:
        # This causes problem when man_len < FP16_MAX_LEN,
        # which shouldn't happen in this project, ignore that
        man = [1] + man + [0]*(man_len - FP16_MAN_LEN - 1)
    
    # Save new EXP
    exp = int2bin(expn, exp_len)
    if(DEBUGMODE):
        print(sgn, numdstr(exp), numdstr(man))
        
    return [int(sgn)] + exp + man
    
    
def fpadd_fmt(fmt, packa, packb):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    # Unpack Data
    sgna = packa[0]
    expa = packa[1:exp_len+1]
    mana = packa[exp_len+1:]
    
    sgnb = packb[0]
    expb = packb[1:exp_len+1]
    manb = packb[exp_len+1:]
 
    #          ea > eb     ea < eb
    # sa sb  sy    my     sy    my
    #  0  0   0  ma + mb   0  ma + mb
    #  0  1   0  ma - mb   1  mb - ma
    #  1  0   1  ma - mb   0  mb - ma
    #  1  1   1  ma + mb   1  ma + mb
    #
    # addsub: ma + mb: 0
    #         ma - mb: 1
    #         mb - ma: 2
    # 
    # This implementation is not very elegant, 
    # you should be able to simply subtract and flip negative values
    # Then, the flags are modified accordingly
    #
    # I did this trying to reduce bit-flipping (to reduce power)
    # But the inversion in the subtractor is doing bit-flipping anyway.
    
    if(DEBUGMODE):
        print("A: ", sgna, numdstr(expa), numdstr(mana))
        print("B: ", sgnb, numdstr(expb), numdstr(manb))
    # - PIPELINE STEP 1: exp compare
    expa_sub_b = bin2int(expa) - bin2int(expb)
    expa_lt_b = (expa_sub_b > 0)
    
    if (sgna==sgnb):
        sign_diff = 0;
        sgny = sgna
        addsub = 0
    elif (sgnb==1):
        sign_diff = 1;
        if(expa_lt_b):
            sgny = 0
            addsub = 1
        else:
            sgny = 1
            addsub = 2
    else:
        sign_diff = 1;
        if(expa_lt_b):
            sgny = 1
            addsub = 1
        else:
            sgny = 0
            addsub = 2
            
    
    # - PIPELINE STEP 2 (probably 2-stages): shift
    # Shift the smaller value to the larger value
    if(expa_lt_b):
        expy = expa
        manb = ([0]*(abs(expa_sub_b)) + manb)[0:len(manb)];
    else:
        expy = expb
        mana = ([0]*(abs(expa_sub_b)) + mana)[0:len(mana)];

    # - PIPELINE STEP 3 (probably 2-stages): Sum
    if(addsub==0):
        many = int2bin(bin2int(mana)+bin2int(manb),len(mana)+1)
    elif(addsub==1):
        many = int2bin(bin2int(mana)-bin2int(manb),len(mana)+1)
    else:
        many = int2bin(bin2int(manb)-bin2int(mana),len(mana)+1)
        
    if(DEBUGMODE):
        print("Addsub: ",addsub)
    if(DEBUGMODE):
        print("A: ", sgna, numdstr(expa), numdstr(mana))
        print("B: ", sgnb, numdstr(expb), numdstr(manb))
        print("Y: ", sgny, numdstr(expy), numdstr(many), "Before ADJ")
        
    # - PIPELINE STEP 4: Normalize - detect
    
    if(is_all(many,0)):
        # Zero
        return [0] + [0]*exp_len + [0]*man_len
    # Find the initial one
    many_zerocount = len(many) - len(numdstr(many).lstrip('0'));
    
    if(DEBUGMODE):
        print("Zero Count:", many_zerocount)
    # - PIPELINE STEP 5: Normalize - Shift
    # normalize many by shifiting it to the left
    many = (many+[0]*many_zerocount)[many_zerocount+1:len(many)-1]
    if(sign_diff):
        expyn = bin2int(expy)+many_zerocount+1
    else:
        expyn = bin2int(expy)+many_zerocount+2
    expy = int2bin(expyn,len(expy))

        
    if(DEBUGMODE):
        print("Y: ", sgny, numdstr(expy), numdstr(many))
    return [int(sgny)] + expy + many

def fpadd_verif(fpa, fpb, msg):
    fpy = fpa + fpb
    fpa = fp16_to_ifmt(FMT_FP32N,fpa)
    fpb = fp16_to_ifmt(FMT_FP32N,fpb)
    k = fpdecode(FMT_FP32, fpadd_ifmt(FMT_FP32N, fpa, fpb));
    if(k==fpy):
        print('- pass -', msg)
    else:
        print('- FAIL - ', msg)
    print("Expected:  ", fpy);
    print("Actual:    ", k)

fpadd_verif(-1,1,'Unity')
fpadd_verif(2,0,'Add Positive With Zero')
fpadd_verif(-2,0,'Add Negative With Zero')
fpadd_verif(-0.5,1/3,'Infinite Fractal')
# Known Issue: 0 + (-2)  or (-2) + 0 = -1
# 0 and other DENORM requries special attention...

ALU:
A: 0010000000000000000000000
B: 1110000000000000000000000
after ALU: exp: 01111111 many: 0000000000000000000000000
fpdecode: DENORM
- pass - Unity
Expected:   0
Actual:     0.0
ALU:
A: 0010000000000000000000000
B: 0000000000000000000000000
after ALU: exp: 10000000 many: 0010000000000000000000000
many_zerocount: 2
after SFT: exp: 10000000 many: 000000000000000000000000
- pass - Add Positive With Zero
Expected:   2
Actual:     2.0
ALU:
A: 0010000000000000000000000
B: 0000000000000000000000000
after ALU: exp: 10000000 many: 0010000000000000000000000
many_zerocount: 2
after SFT: exp: 10000000 many: 000000000000000000000000
- pass - Add Negative With Zero
Expected:   -2
Actual:     -2.0
ALU:
A: 0010000000000000000000000
B: 1110101010101100000000000
after ALU: exp: 01111110 many: 0000101010101100000000000
many_zerocount: 4
after SFT: exp: 01111100 many: 101010110000000000000000
- FAIL -  Infinite Fractal
Expected:   -0.16666666666666669
Actual:     -0.1669921875


In [316]:

fp16_to_internal_fmt(FMT_FP32N, -0.5)
fp16_to_internal_fmt(FMT_FP32N, -1/3)
# Largest Denorm Number
numdstr(fp16_to_internal_fmt(FMT_FP32N, 0.000060975552))

'00111000011111111110000000000000'

In [478]:
def fpadd_ifmt(fmt, packa, packb):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    # Unpack Data
    # Caution: Here we're assuming that all incoming data are in denorm format
    # They have exponents (unlike IEEE terminalogy), but also have an explicit 1
    sgna = packa[0]
    expa = packa[1:exp_len+1]
    mana = [0]*2 + packa[exp_len+1:]
    
    sgnb = packb[0]
    expb = packb[1:exp_len+1]
    manb = [0]*2 + packb[exp_len+1:]
    
    if(DEBUGMODE):
        print("A: ", sgna, numdstr(expa), numdstr(mana))
        print("B: ", sgnb, numdstr(expb), numdstr(manb))
    
    # - PIPELINE STEP 1: Exponent Compare 
    expa_sub_b = bin2int(expa) - bin2int(expb)
    expa_gt_b = (expa_sub_b > 0)
    
    # Addsub Control Signals:
    #          ea > eb     ea < eb
    #            SRB         SRA
    # sa sb  sy    my     sy    my
    #  0  0   0  ma + mb   0  mb + ma
    #  0  1   0  ma - mb   1  mb - ma
    #  1  0   1  ma - mb   0  mb - ma
    #  1  1   1  ma + mb   1  mb + ma
    #
    # addsub: ma + mb: 0
    #         ma - mb: 1
    #         mb - ma: 2
    
    ctrl_manxch = -expa_gt_b         # Exchange ALU Inputs
    ctrl_mansub = xor(sgna,sgnb)     # Subtract Mode
    if(expa_gt_b):
        sgny = sgna;
    else:
        sgny = sgnb;
        
    # - PIPELINE STEP 2: Shift
    # Shifter
    if(expa_gt_b):
        expy = expa
        manb = ([0]*(abs(expa_sub_b)) + manb)[0:len(manb)];
    else:
        expy = expb
        mana = ([0]*(abs(expa_sub_b)) + mana)[0:len(mana)];
    
    # - PIPELINE STEP 3: Addsub
    # ALU MUX
    if expa_gt_b:
        alua = mana
        alub = manb
    else:
        alua = manb
        alub = mana

    if ctrl_mansub:
        alub = binneg(alub)
    many = binadd(alua,alub,0);
    many = many[1:]
    print("ALU:")
    print("A:", numdstr(alua))
    print("B:", numdstr(alub))
    print("after ALU: exp:",numdstr(expy),"many:", numdstr(many));
    
    # - PIPELINE STEP 4: Normalize - detect
    if(is_all(many,0)):
        # Zero
        return [0] + [0]*exp_len + [0]*man_len
    
    # Find the initial one
    many_zerocount = len(many) - len(numdstr(many).lstrip('0'));
    
    # - PIPELINE STEP 5: Normalize - Shift
    
    many = (many+[0]*(many_zerocount+1))[many_zerocount:]
    expyn = bin2int(expy)+2-many_zerocount
    expy = int2bin(expyn,len(expy))
    print("many_zerocount:", many_zerocount)
    print("after SFT: exp:",numdstr(expy),"many:", numdstr(many));
    
    # Deal with denorm
    if(is_all(expy,0)):
        print("Denorm")
        return [int(sgny)] + [0]*exp_len + many[0:man_len]
    else:
        a = numdstr([int(sgny)] + expy + many[1:]);
        print(a[0],a[1:FP32_EXP_LEN+1],a[FP32_EXP_LEN+1:])
        return [int(sgny)] + expy + many[1:]
    
def fpadd_verif(fpa, fpb, msg):
    fpy = fpa + fpb
    fpa = fp16_to_ifmt(FMT_FP32N,fpa)
    fpb = fp16_to_ifmt(FMT_FP32N,fpb)
    k = fpdecode(FMT_FP32, fpadd_ifmt(FMT_FP32N, fpa, fpb));
    if(k==fpy):
        print('- pass -', msg)
    else:
        print('- FAIL - ', msg)
    print("Expected:  ", fpy);
    print("Actual:    ", k)

fpadd_verif(-1,1,'Unity')
fpadd_verif(2,0,'Add Positive With Zero')
fpadd_verif(-2,0,'Add Negative With Zero')
ta = 0.1;
tb = -3;
fpadd_verif(ta,tb,'Infinite Fractal')
a = numdstr(fp16_to_internal_fmt(FMT_FP32N, ta + tb))
print(a[0],a[1:FP32_EXP_LEN+1],a[FP32_EXP_LEN+1:])
# Known Issue: 0 + (-2)  or (-2) + 0 = -1
# 0 and other DENORM requries special attention...

ALU:
A: 0010000000000000000000000
B: 1110000000000000000000000
after ALU: exp: 01111111 many: 0000000000000000000000000
fpdecode: DENORM
- pass - Unity
Expected:   0
Actual:     0.0
ALU:
A: 0010000000000000000000000
B: 0000000000000000000000000
after ALU: exp: 10000000 many: 0010000000000000000000000
many_zerocount: 2
after SFT: exp: 10000000 many: 10000000000000000000000000
0 10000000 0000000000000000000000000
- FAIL -  Add Positive With Zero
Expected:   2
Actual:     8.0
ALU:
A: 0010000000000000000000000
B: 0000000000000000000000000
after ALU: exp: 10000000 many: 0010000000000000000000000
many_zerocount: 2
after SFT: exp: 10000000 many: 10000000000000000000000000
1 10000000 0000000000000000000000000
- FAIL -  Add Negative With Zero
Expected:   -2
Actual:     -8.0
ALU:
A: 0011000000000000000000000
B: 1111111001100110100000000
after ALU: exp: 10000000 many: 0010111001100110100000000
many_zerocount: 2
after SFT: exp: 10000000 many: 10111001100110100000000000
1 10000000 01110011001101000

1 01111111 10000000000000000000000
0 01111111 10000000000000000000000
A:  1 01111111 0010000000000000000000000
B:  0 01111111 0010000000000000000000000
ALU:
A: 0010000000000000000000000
B: 1110000000000000000000000
after ALU: many:  0000000000000000000000000
fpdecode: DENORM
- pass - Unity
Expected:   0
Actual:     0.0
0 10000000 10000000000000000000000
DENORM
0 01110000 00000000000000000000000
A:  0 10000000 0010000000000000000000000
B:  0 01110000 0000000000000000000000000
ALU:
A: 0010000000000000000000000
B: 0000000000000000000000000
after ALU: many:  0010000000000000000000000
after SFT: many:  0000000000000000000000000
- pass - Add Positive With Zero
Expected:   2
Actual:     2.0
1 10000000 10000000000000000000000
DENORM
0 01110000 00000000000000000000000
A:  1 10000000 0010000000000000000000000
B:  0 01110000 0000000000000000000000000
ALU:
A: 0010000000000000000000000
B: 0000000000000000000000000
after ALU: many:  0010000000000000000000000
after SFT: many:  00000000000000000000000